In [1]:
DIR = "norms"

import os
import emcee
import reddemcee

import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from scipy import stats
from pyazr import azure2
from multiprocess import Pool, current_process

# Restrict processes to one thread only
os.environ['OMP_NUM_THREADS'] = '1'

# emcee variables
nsteps = 10000 # How many steps should each walker take?
nthin  = 10    # How often should the walker save a step?
nprocs = 6     # How many Python processes do you want to allocate?
ntemps = 10

# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

# Define the parameters prior distributions
priors = [
    stats.norm(1.63,0.12),

    stats.uniform(2.30, 0.10), stats.uniform(0, 1e6), stats.uniform(-10, 20),
    stats.uniform(-1e8,2e8),

    stats.uniform(3.45, 0.10), stats.uniform(0, 1e6), stats.uniform(-10, 20), stats.uniform(-10, 20),
    stats.uniform(-1e8,2e8), stats.uniform(-1e8,2e8),

    stats.uniform(3.50, 0.10), stats.uniform(0, 1e6),
    
    stats.lognorm(0.05),
    stats.lognorm(0.05),
    stats.lognorm(0.05),
    stats.lognorm(0.069),
    stats.lognorm(0.079),
    stats.lognorm(0.10),
    stats.lognorm(0.10),
    stats.lognorm(0.10),
    stats.lognorm(0.10),
    stats.lognorm(0.10),
    stats.uniform(0, 100),
    stats.uniform(0, 100),
    stats.uniform(0, 100),
    stats.uniform(0, 100),
    stats.uniform(0, 100)
]

In [2]:
# We read the .azr file and set the external capture file to speed up the calculation
azr = azure2('12c_pg.azr', nprocs=nprocs)

In [3]:
# We get the initial values from AZURE2
theta0 = azr.params
ntheta = len(theta0)

# We'll read the data from the output file since it's already in the center-of-mass frame
y = azr.cross
yerr = azr.cross_err
ndata = sum( len(segment) for segment in y )

In [4]:
# Prior log probability
def lnPi( theta ):
    return np.sum([pi.logpdf(t) for (pi, t) in zip(priors, theta)])

# Log likelihood
def lnl( theta ):
    res = 0
    try: proc = int(current_process().name.split('-')[1]) - 1
    except: proc = 0
    mu = azr.calculate( theta[:ntheta], proc=proc )
    for i in range( len( mu ) ):
        idx = ntheta + i
        res += np.sum( -0.5 * np.log(2 * np.pi * pow(yerr[i], 2) ) - 0.5 * pow((mu[i] - y[i] * theta[idx]) / (yerr[i] * theta[idx]), 2) )
    return res

# Posterior log probability
def lnP( theta ):
    proc = int(current_process().name.split('-')[1]) - 1
    lnpi = lnPi( theta )
    if not np.isfinite( lnpi ): return -np.inf
    return lnl( theta, proc=proc ) + lnpi

In [5]:
# Get some good guesses for the initial walker positions
data = []
files = glob( "results/{}/samples_*.txt".format(DIR) )
for file in files:
    data.append( np.loadtxt( file ) )
data = np.concatenate( data, axis=0 )
mean = np.mean( data, axis=0 )

nw = 2 * len(priors)
p0 = np.zeros( (ntemps,nw,len(priors)) )
for t in range(ntemps):
    for i in range(nw):
        for j in range(len(priors)):
            if( j not in [1, 5, 11] ): p0[t,i,j] = np.sign(mean[j]) * stats.uniform( abs(mean[j]) * 0.5, abs(mean[j]) * 1.0 ).rvs()
            else: p0[t,i,j] = np.sign(mean[j]) * stats.uniform( abs(mean[j]) * 0.99, abs(mean[j]) * 0.02 ).rvs()

In [6]:
backends = []
for t in range(ntemps):
    backends.append( emcee.backends.HDFBackend('results/ptemcee/samples_{}.h5'.format(t)) )
    backends[t].reset(nw, len(priors))

with Pool( nprocs ) as pool:
    sampler = reddemcee.PTSampler( nw, len(priors), lnl, lnPi, ntemps=ntemps, pool=pool, backend=backends)
    sampler.run_mcmc(list(p0), nsteps)

100%|██████████| 1000/1000 [1:01:23<00:00,  3.68s/it]
